In [ ]:
import numpy as np
import tensorflow as tf
import joblib
print("done")

In [ ]:
# Load base models
model1 = tf.keras.models.load_model("mlp1.keras")  # NO2 & O3
model2 = tf.keras.models.load_model("mlp2.keras")  # PM2.5
model3 = tf.keras.models.load_model("mlp3.keras")  # PM10
model4 = tf.keras.models.load_model("mlp4.keras")  # SO2 & CO

print("✅ Models Loaded Successfully!")


In [ ]:
import numpy as np

# 🚀 New Weight Distribution (More emphasis on harmful pollutants)
weights = np.array([0.10, 0.35, 0.30, 0.25])  # Reduce NO₂ & O₃ impact

# Breakdown:
# NO₂ & O₃  →  15%  (Still important, but less impactful than PM2.5)
# PM2.5     →  40%  (Most impactful, should have the highest weight)
# PM10      →  30%  (Slightly increased to reflect its effect)
# SO₂ & CO  →  25%  (CO is dangerous, needs a higher weight)


In [ ]:
import numpy as np

# Define min and max values based on air quality standards
MIN_VALUES = np.array([0, 0, 0, 0, 0, 0])  # Minimum pollutant levels (µg/m³)
MAX_VALUES = np.array([200, 200, 500, 500, 100, 50])  # Maximum observed values

def get_user_inputs():
    while True:
        try:
            no2 = float(input("Enter NO₂ level (µg/m³): "))
            o3 = float(input("Enter O₃ level (µg/m³): "))
            pm25 = float(input("Enter PM2.5 level (µg/m³): "))
            pm10 = float(input("Enter PM10 level (µg/m³): "))
            so2 = float(input("Enter SO₂ level (µg/m³): "))
            co = float(input("Enter CO level (mg/m³): "))  # CO usually in mg/m³
            
            # Convert CO from mg/m³ to µg/m³ (1 mg/m³ = 1000 µg/m³)
            co *= 1000  
            
            # Store in an array
            raw_values = np.array([no2, o3, pm25, pm10, so2, co])

            # Min-Max Scaling
            scaled_values = (raw_values - MIN_VALUES) / (MAX_VALUES - MIN_VALUES)
            
            return scaled_values
        
        except ValueError:
            print("❌ Invalid input! Please enter numerical values only.")

# Get scaled inputs
scaled_inputs = get_user_inputs()


In [ ]:
# Format inputs correctly for the models
input1 = np.array([[scaled_inputs[0], scaled_inputs[1]]])  # Model 1: NO₂ & O₃
input2 = np.array([[scaled_inputs[2]]])  # Model 2: PM2.5
input3 = np.array([[scaled_inputs[3]]])  # Model 3: PM10
input4 = np.array([[scaled_inputs[4], scaled_inputs[5]]])  # Model 4: SO₂ & CO

print("\n✅ Scaled Inputs Ready for Prediction!")


In [ ]:
# Get predictions from base models (each outputs (1,6) softmax probabilities)
pred_1 = model1.predict(input1)  
pred_2 = model2.predict(input2)  
pred_3 = model3.predict(input3)  
pred_4 = model4.predict(input4)  

print("✅ Predictions Obtained from Base Models!")

# 📌 Print the outputs of each model
print("\n✅ Predictions Obtained from Base Models!\n")
print(f"📊 Model 1 Predictions: {pred_1}")
print(f"📊 Model 2 Predictions: {pred_2}")
print(f"📊 Model 3 Predictions: {pred_3}")
print(f"📊 Model 4 Predictions: {pred_4}")


In [ ]:
# Stack predictions vertically => shape will be (4, 6)
stacked_preds = np.vstack([pred_1, pred_2, pred_3, pred_4])  # Shape: (4, 6)

# Define weights (must match number of rows, i.e., 4)
weights = [0.10, 0.35, 0.30, 0.25]  # Adjust as needed

# Compute weighted average along axis=0 (i.e., average each class score across models)
weighted_preds = np.average(stacked_preds, axis=0, weights=weights)  # Shape: (6,)

print("Weighted Predictions:", weighted_preds)
print("Done")

In [ ]:
print(f"🔢 Weighted Predictions: {weighted_preds}")


In [ ]:
# Define air quality labels (adjusted to 0-based indexing)
air_quality_labels = {
    0: "Very Good",
    1: "Good",
    2: "Satisfactory",
    3: "Borderline",
    4: "Poor",
    5: "Very Poor"
}

# Get final predicted category (ensure 1D array)
final_category = np.argmax(weighted_preds.flatten())  # Use .flatten() for safety

# Print final prediction
print(f"Air Quality Classification : {air_quality_labels[final_category]}")
